## Kombinasi 3 :
- Delete duplicate
- Impute missing value using iterative imputer
- Outlier capping with winsorization
- Encoding
- Standard Scaler
- Feature selection with Decision Tree

In [1]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv("../UFC_train.csv")
df_test = pd.read_csv('../regression/New_UFC_Test_Regression_X(3).csv')
df.head()

,R_fighter,B_fighter,Referee,date,location,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,Winner
0,Joe Riggs,Joe Doerksen,Steve Mazzagatti,2004-08-21,"Las Vegas, Nevada, USA",False,Middleweight,NaN,NaN,NaN,...,0,0,0,Southpaw,182.88,177.80,185.0,26.0,21.0,Red
1,Jorge Masvidal,Al Iaquinta,Keith Peterson,2015-04-04,"Fairfax, Virginia, USA",False,Lightweight,1.156250,0.000000,0.394141,...,0,1,0,Orthodox,180.34,187.96,170.0,27.0,30.0,Blue
2,Dan Stittgen,Stephen Thompson,Josh Rosenthal,2012-02-04,"Las Vegas, Nevada, USA",False,Welterweight,NaN,NaN,NaN,...,0,0,0,Orthodox,185.42,NaN,170.0,28.0,31.0,Blue
3,Josh Koscheck,Johny Hendricks,Kevin Mulhall,2012-05-05,"East Rutherford, New Jersey, USA",False,Welterweight,0.695312,0.000000,0.783359,...,6,3,0,Orthodox,177.80,185.42,170.0,28.0,34.0,Blue
4,John Dodson,Manvel Gamburyan,James Warring,2016-04-16,"Tampa, Florida, USA",False,Bantamweight,0.500000,0.266602,0.381462,...,3,0,1,Orthodox,160.02,167.64,135.0,34.0,31.0,Red


In [3]:
# Delete duplicate
print(f"Shape before dropping duplicate : {df.shape}")
df.drop_duplicates(inplace=True)
print(f"Shape after dropping duplicate : {df.shape}")

Shape before dropping duplicate : (5410, 144)
Shape after dropping duplicate : (5410, 144)


In [4]:
def check_null(df):
    col_na = df.isnull().sum().sort_values(ascending=True)
    percent = col_na / len(df)
    missing_data = pd.concat([col_na, percent], axis=1, keys=['Total', 'Percent'])

    if (missing_data[missing_data['Total'] > 0].shape[0] == 0):
        print("Tidak ditemukan missing value pada dataset")
    else:
        print(missing_data[missing_data['Total'] > 0])

In [5]:
check_null(df)

                         Total   Percent
R_Weight_lbs                 2  0.000370
R_Height_cms                 4  0.000739
B_Weight_lbs                 8  0.001479
B_Height_cms                10  0.001848
R_Stance                    27  0.004991
...                        ...       ...
B_avg_GROUND_landed       1293  0.239002
B_avg_GROUND_att          1293  0.239002
B_avg_opp_CLINCH_landed   1293  0.239002
B_avg_TD_att              1293  0.239002
B_avg_HEAD_landed         1293  0.239002

[109 rows x 2 columns]


In [6]:
# Split string and number
# Split dataframe into object type and number type
df_object = df.select_dtypes(include='object')
df_number = df.select_dtypes(include=np.number)

df_test_object = df_test.select_dtypes(include='object')
df_test_number = df_test.select_dtypes(include=np.number)

In [7]:
# Impute missing value using iterative imputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.tree import ExtraTreeRegressor

imputer = IterativeImputer(initial_strategy='median', random_state=42, estimator=ExtraTreeRegressor(random_state=42), max_iter=15)
df_imputed_number = imputer.fit_transform(df_number)
df_imputed_number = pd.DataFrame(df_imputed_number, columns=df_number.columns)

df_test_imputed_number = imputer.fit_transform(df_test_number)
df_test_imputed_number = pd.DataFrame(df_test_imputed_number, columns=df_test_number.columns)

c:\Python311\Lib\site-packages\sklearn\impute\_iterative.py:796: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\impute\_iterative.py:796: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [8]:
# Drop R_fighter, B_fighter, Referee, location, and date
df_object = df_object.drop(['Winner'], axis=1)

In [9]:
# Combine df_object and df_imputed_number
df = pd.concat([df_object, df_imputed_number], axis=1)
df_test = pd.concat([df_test_object, df_test_imputed_number], axis=1)
df.head()

,R_fighter,B_fighter,Referee,date,location,weight_class,B_Stance,R_Stance,B_avg_KD,B_avg_opp_KD,...,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Joe Riggs,Joe Doerksen,Steve Mazzagatti,2004-08-21,"Las Vegas, Nevada, USA",Middleweight,Orthodox,Southpaw,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,182.88,177.80,185.0,26.0,21.0
1,Jorge Masvidal,Al Iaquinta,Keith Peterson,2015-04-04,"Fairfax, Virginia, USA",Lightweight,Orthodox,Orthodox,1.156250,0.000000,...,0.0,4.0,0.0,1.0,0.0,180.34,187.96,170.0,27.0,30.0
2,Dan Stittgen,Stephen Thompson,Josh Rosenthal,2012-02-04,"Las Vegas, Nevada, USA",Welterweight,Orthodox,Orthodox,0.500000,0.000000,...,0.0,0.0,0.0,0.0,0.0,185.42,185.42,170.0,28.0,31.0
3,Josh Koscheck,Johny Hendricks,Kevin Mulhall,2012-05-05,"East Rutherford, New Jersey, USA",Welterweight,Southpaw,Orthodox,0.695312,0.000000,...,1.0,5.0,6.0,3.0,0.0,177.80,185.42,170.0,28.0,34.0
4,John Dodson,Manvel Gamburyan,James Warring,2016-04-16,"Tampa, Florida, USA",Bantamweight,Orthodox,Orthodox,0.500000,0.266602,...,0.0,2.0,3.0,0.0,1.0,160.02,167.64,135.0,34.0,31.0


In [10]:
check_null(df)

          Total   Percent
R_Stance     27  0.004991
Referee      31  0.005730
B_Stance     64  0.011830


In [11]:
# Impute missing R_stance and B_stance with mode
df['R_Stance'].fillna(df['R_Stance'].mode()[0], inplace=True)
df['B_Stance'].fillna(df['B_Stance'].mode()[0], inplace=True)

df_test['R_Stance'].fillna(df_test['R_Stance'].mode()[0], inplace=True)
df_test['B_Stance'].fillna(df_test['B_Stance'].mode()[0], inplace=True)

In [12]:
check_null(df)

         Total  Percent
Referee     31  0.00573


In [13]:
def check_outlier(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)

    # Menghitung RUB dan RLB.
    IQR = Q3 - Q1
    lower_limit = Q1 - 1.5*IQR
    upper_limit = Q3 + 1.5*IQR

    # Menampilkan banyaknya outlier pada atribut.
    outliers = (df < lower_limit) | (df > upper_limit)
    print ("Outlier pada tiap atribut:")
    print(outliers.sum())

    return outliers

In [14]:
# Split string and number
# Split dataframe into object type and number type
df_object = df.select_dtypes(include='object')
df_number = df.select_dtypes(include=np.number)

In [15]:
outliers = dict(check_outlier(df_number).sum())
print("\n\npercentage of outliers in each column:")
for key in outliers.keys():
    print(f"{key} = {outliers[key]/df_number[key].shape[0] * 100}%")

Outlier pada tiap atribut:
B_avg_KD                 476
B_avg_opp_KD             646
B_avg_SIG_STR_pct        171
B_avg_opp_SIG_STR_pct    143
B_avg_TD_pct               0
                        ... 
R_Height_cms              19
R_Reach_cms               85
R_Weight_lbs             301
B_age                     31
R_age                     75
Length: 134, dtype: int64


percentage of outliers in each column:
B_avg_KD = 8.79852125693161%
B_avg_opp_KD = 11.940850277264325%
B_avg_SIG_STR_pct = 3.1608133086876156%
B_avg_opp_SIG_STR_pct = 2.6432532347504623%
B_avg_TD_pct = 0.0%
B_avg_opp_TD_pct = 0.0%
B_avg_SUB_ATT = 6.506469500924214%
B_avg_opp_SUB_ATT = 6.691312384473198%
B_avg_REV = 17.689463955637706%
B_avg_opp_REV = 17.13493530499076%
B_avg_SIG_STR_att = 3.3086876155268024%
B_avg_SIG_STR_landed = 2.7356746765249538%
B_avg_opp_SIG_STR_att = 3.6229205175600745%
B_avg_opp_SIG_STR_landed = 2.9944547134935307%
B_avg_TOTAL_STR_att = 1.77449168207024%
B_avg_TOTAL_STR_landed = 1.7190388170055

In [16]:
from scipy.stats.mstats import winsorize

from scipy.stats.mstats import winsorize

# Identify columns with more than 0.5% outliers
outlier_columns = []
for key in outliers.keys():
    if outliers[key] / df_number[key].shape[0] > 0.005:
        outlier_columns.append(key)

# Winsorize the identified columns
for column in outlier_columns:
    df_number[column] = winsorize(df_number[column], limits=(0, 0.1))

# Combine df_object and df_number
df = pd.concat([df_object, df_number], axis=1)
df.head()

,R_fighter,B_fighter,Referee,date,location,weight_class,B_Stance,R_Stance,B_avg_KD,B_avg_opp_KD,...,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Joe Riggs,Joe Doerksen,Steve Mazzagatti,2004-08-21,"Las Vegas, Nevada, USA",Middleweight,Orthodox,Southpaw,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,182.88,177.80,185.0,26.0,21.0
1,Jorge Masvidal,Al Iaquinta,Keith Peterson,2015-04-04,"Fairfax, Virginia, USA",Lightweight,Orthodox,Orthodox,0.750000,0.000000,...,0.0,4.0,0.0,1.0,0.0,180.34,187.96,170.0,27.0,30.0
2,Dan Stittgen,Stephen Thompson,Josh Rosenthal,2012-02-04,"Las Vegas, Nevada, USA",Welterweight,Orthodox,Orthodox,0.500000,0.000000,...,0.0,0.0,0.0,0.0,0.0,185.42,185.42,170.0,28.0,31.0
3,Josh Koscheck,Johny Hendricks,Kevin Mulhall,2012-05-05,"East Rutherford, New Jersey, USA",Welterweight,Southpaw,Orthodox,0.695312,0.000000,...,1.0,4.0,4.0,2.0,0.0,177.80,185.42,170.0,28.0,34.0
4,John Dodson,Manvel Gamburyan,James Warring,2016-04-16,"Tampa, Florida, USA",Bantamweight,Orthodox,Orthodox,0.500000,0.266602,...,0.0,2.0,3.0,0.0,0.0,160.02,167.64,135.0,34.0,31.0


In [17]:
outliers = dict(check_outlier(df_number).sum())
print("\n\npercentage of outliers in each column:")
for key in outliers.keys():
    print(f"{key} = {outliers[key]/df_number[key].shape[0] * 100}%")

Outlier pada tiap atribut:
B_avg_KD                   0
B_avg_opp_KD             646
B_avg_SIG_STR_pct         74
B_avg_opp_SIG_STR_pct     54
B_avg_TD_pct               0
                        ... 
R_Height_cms              19
R_Reach_cms               35
R_Weight_lbs               0
B_age                      0
R_age                      5
Length: 134, dtype: int64


percentage of outliers in each column:
B_avg_KD = 0.0%
B_avg_opp_KD = 11.940850277264325%
B_avg_SIG_STR_pct = 1.3678373382624769%
B_avg_opp_SIG_STR_pct = 0.9981515711645103%
B_avg_TD_pct = 0.0%
B_avg_opp_TD_pct = 0.0%
B_avg_SUB_ATT = 0.0%
B_avg_opp_SUB_ATT = 0.0%
B_avg_REV = 17.689463955637706%
B_avg_opp_REV = 17.13493530499076%
B_avg_SIG_STR_att = 0.0%
B_avg_SIG_STR_landed = 0.0%
B_avg_opp_SIG_STR_att = 0.0%
B_avg_opp_SIG_STR_landed = 0.0%
B_avg_TOTAL_STR_att = 0.0%
B_avg_TOTAL_STR_landed = 0.0%
B_avg_opp_TOTAL_STR_att = 0.0%
B_avg_opp_TOTAL_STR_landed = 0.0%
B_avg_TD_att = 0.0%
B_avg_TD_landed = 0.0%
B_avg_opp_TD_att

c:\Python311\Lib\site-packages\numpy\lib\function_base.py:4823: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(
c:\Python311\Lib\site-packages\numpy\lib\function_base.py:4823: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(


In [18]:
df['gender'] = df['weight_class'].apply(lambda x: 'women' if 'women' in x.lower() else 'male')
df_test['gender'] = df_test['weight_class'].apply(lambda x: 'women' if 'women' in x.lower() else 'male')

In [19]:
df.head()

,R_fighter,B_fighter,Referee,date,location,weight_class,B_Stance,R_Stance,B_avg_KD,B_avg_opp_KD,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,gender
0,Joe Riggs,Joe Doerksen,Steve Mazzagatti,2004-08-21,"Las Vegas, Nevada, USA",Middleweight,Orthodox,Southpaw,0.000000,0.000000,...,0.0,0.0,0.0,0.0,182.88,177.80,185.0,26.0,21.0,male
1,Jorge Masvidal,Al Iaquinta,Keith Peterson,2015-04-04,"Fairfax, Virginia, USA",Lightweight,Orthodox,Orthodox,0.750000,0.000000,...,4.0,0.0,1.0,0.0,180.34,187.96,170.0,27.0,30.0,male
2,Dan Stittgen,Stephen Thompson,Josh Rosenthal,2012-02-04,"Las Vegas, Nevada, USA",Welterweight,Orthodox,Orthodox,0.500000,0.000000,...,0.0,0.0,0.0,0.0,185.42,185.42,170.0,28.0,31.0,male
3,Josh Koscheck,Johny Hendricks,Kevin Mulhall,2012-05-05,"East Rutherford, New Jersey, USA",Welterweight,Southpaw,Orthodox,0.695312,0.000000,...,4.0,4.0,2.0,0.0,177.80,185.42,170.0,28.0,34.0,male
4,John Dodson,Manvel Gamburyan,James Warring,2016-04-16,"Tampa, Florida, USA",Bantamweight,Orthodox,Orthodox,0.500000,0.266602,...,2.0,3.0,0.0,0.0,160.02,167.64,135.0,34.0,31.0,male


In [20]:
df['weight_class'] = df['weight_class'].str.replace('Women', '')
df_test['weight_class'] = df_test['weight_class'].str.replace('Women', '')

In [21]:
# Get all weight_class values
weight_class = df['weight_class'].unique()
weight_class

array(['Middleweight', 'Lightweight', 'Welterweight', 'Bantamweight',
       'Flyweight', 'LightHeavyweight', 'Strawweight', 'Featherweight',
       'OpenWeight', 'Heavyweight', 'CatchWeight'], dtype=object)

In [22]:
# Create a dictionary to map weight_class values to numbers
weight_class_dict = {
    'CatchWeight' : 0,
    'Strawweight' : 1,
    'Flyweight' : 2,
    'Bantamweight' : 3,
    'Featherweight' : 4,
    'Lightweight' : 5,
    'Welterweight' : 6,
    'Middleweight' : 7,
    'LightHeavyweight' : 8,
    'Heavyweight' : 9,
    'OpenWeight' : 10,
}

gender_dict = {
    'male' : 1,
    'women' : 0
}

In [23]:
# Map each weight_class value to the correct number
df['weight_class'] = df['weight_class'].map(weight_class_dict)
df_test['weight_class'] = df_test['weight_class'].map(weight_class_dict)

In [24]:
# Map each gender value to the correct number
df['gender'] = df['gender'].map(gender_dict)
df_test['gender'] = df_test['gender'].map(gender_dict)

In [25]:
df.head()

,R_fighter,B_fighter,Referee,date,location,weight_class,B_Stance,R_Stance,B_avg_KD,B_avg_opp_KD,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,gender
0,Joe Riggs,Joe Doerksen,Steve Mazzagatti,2004-08-21,"Las Vegas, Nevada, USA",7,Orthodox,Southpaw,0.000000,0.000000,...,0.0,0.0,0.0,0.0,182.88,177.80,185.0,26.0,21.0,1
1,Jorge Masvidal,Al Iaquinta,Keith Peterson,2015-04-04,"Fairfax, Virginia, USA",5,Orthodox,Orthodox,0.750000,0.000000,...,4.0,0.0,1.0,0.0,180.34,187.96,170.0,27.0,30.0,1
2,Dan Stittgen,Stephen Thompson,Josh Rosenthal,2012-02-04,"Las Vegas, Nevada, USA",6,Orthodox,Orthodox,0.500000,0.000000,...,0.0,0.0,0.0,0.0,185.42,185.42,170.0,28.0,31.0,1
3,Josh Koscheck,Johny Hendricks,Kevin Mulhall,2012-05-05,"East Rutherford, New Jersey, USA",6,Southpaw,Orthodox,0.695312,0.000000,...,4.0,4.0,2.0,0.0,177.80,185.42,170.0,28.0,34.0,1
4,John Dodson,Manvel Gamburyan,James Warring,2016-04-16,"Tampa, Florida, USA",3,Orthodox,Orthodox,0.500000,0.266602,...,2.0,3.0,0.0,0.0,160.02,167.64,135.0,34.0,31.0,1


In [26]:
# Get the remaining object columns
df_object = df.select_dtypes(include='object')

# Perform one-hot encoding
df_encoded = pd.get_dummies(df_object)

# Combine the encoded columns with the original dataframe
df = pd.concat([df, df_encoded], axis=1)

# Drop the original object columns
df.drop(df_object.columns, axis=1, inplace=True)

df.head()


,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,B_avg_REV,...,B_Stance_Open Stance,B_Stance_Orthodox,B_Stance_Sideways,B_Stance_Southpaw,B_Stance_Switch,R_Stance_Open Stance,R_Stance_Orthodox,R_Stance_Sideways,R_Stance_Southpaw,R_Stance_Switch
0,7,0.000000,0.000000,0.430000,0.443750,1.000000,0.165000,0.00000,1.000000,0.125,...,False,True,False,False,False,False,False,False,True,False
1,5,0.750000,0.000000,0.394141,0.352422,0.239219,0.011484,0.15625,0.132812,0.000,...,False,True,False,False,False,False,True,False,False,False
2,6,0.500000,0.000000,0.452500,0.313125,0.000000,0.000000,0.00000,0.000000,0.000,...,False,True,False,False,False,False,True,False,False,False
3,6,0.695312,0.000000,0.605000,0.185547,0.088281,0.104375,0.09375,0.062500,0.000,...,False,False,False,True,False,False,True,False,False,False
4,3,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,1.37793,0.000244,0.125,...,False,True,False,False,False,False,True,False,False,False


In [27]:
# Get the remaining object columns
df_test_object = df_test.select_dtypes(include='object')

# Perform one-hot encoding
df_test_encoded = pd.get_dummies(df_test_object)

# Combine the encoded columns with the original dataframe
df_test = pd.concat([df_test, df_test_encoded], axis=1)

# Drop the original object columns
df_test.drop(df_test_object.columns, axis=1, inplace=True)

df_test.head()


,weight_class,id,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,...,"location_Wichita, Kansas, USA","location_Winnipeg, Manitoba, Canada","location_Zagreb, Croatia",B_Stance_Orthodox,B_Stance_Southpaw,B_Stance_Switch,R_Stance_Open Stance,R_Stance_Orthodox,R_Stance_Southpaw,R_Stance_Switch
0,1,0.0,0.500000,0.000000,0.530000,0.425000,0.000000,0.160000,0.000000,0.0000,...,False,False,False,True,False,False,False,True,False,False
1,3,1.0,0.000000,0.000000,0.565000,0.435000,0.750000,1.000000,0.500000,0.5000,...,False,False,False,True,False,False,False,True,False,False
2,7,2.0,0.568909,0.437805,0.583355,0.439665,0.004447,0.166195,0.004333,0.2547,...,False,False,False,True,False,False,False,False,True,False
3,2,3.0,0.500000,0.000000,0.435000,0.355000,0.250000,0.250000,2.000000,0.0000,...,False,False,False,False,True,False,False,True,False,False
4,3,4.0,0.000000,0.000000,0.150000,0.440000,0.330000,0.180000,0.000000,0.0000,...,False,False,False,True,False,False,False,False,True,False


In [28]:
df.to_csv('../Without Feature Selection/UFC_kombinasi23_all_features.csv', index=False)
df_test.to_csv('../regression_kaggle/UFC_test_kombinasi23_all_features.csv', index=False)